# Support Vector Machine

In this tutorial section, we use SketchyOpts to train a support vector machine (SVM) classifier. We first introduce some useful notations. 
| Symbol | Description |
| --- | --- |
| $X \in \mathbb{R}^{n \times p}$ | input features (or design matrix) |
| $y \in \{-1, +1\}^{n}$ | target classes |
| $(x_i, y_i) \in \mathbb{R}^p \times \{-1, +1\}$ | $i$<sup>th</sup> training sample |
| $\beta = [\beta_0, \beta_{1}^{\mathsf{T}}] \in \mathbb{R}^{p+1}$ | parameters of the model (including both the weights $\beta_1$ and bias $\beta_0$) |
| $C \geqslant 0$ | penalty parameter for misclassification |

SVM classifies data by maximizing the "margin" around the hyperplane in feature space that separates all data points of one class from those of the other class. The margin, more precisely, is the minimum distance between sample instances and the decision boundary. Often times the dataset we encounter is nonseparable. To address such difficulty and still be able to make classification decisions, we instead allow sample instances to be on the wrong side of the margin (also called soft-margin SVM). This gives rise to the following optimization problem
$$
\begin{aligned}
    \underset{\beta, \, \{\xi_i\}_{i=1}^{n}}{\operatorname{minimize}} \quad& \frac{1}{2} \lVert \beta_1 \rVert_2^2 + C \, \sum_{i=1}^{n} \xi_i \\
    \text{subject to} \quad& y_i(x_i^{\mathsf{T}}\beta_1 + \beta_0) \geqslant 1 - \xi_i, \, \forall i \\
    & \xi_i \geqslant 0, \, \forall i
\end{aligned}
$$
Here $(\xi_1, \cdots, \xi_n)$ are slack variables characterizing the amount by which the prediction $x_i^{\mathsf{T}}\beta_1 + \beta_0$ is on the wrong side of its margin. 

- When $\xi_i = 0$, the prediction is outside the margin on the correct side of the separating hyperplane;
- when $0 < \xi_i \leqslant 1$, the prediction falls within the margin on the correct side of the separating hyperplane;
- when $\xi_i > 1$, the prediction lies on the wrong side of the separating hyperplane. 

The objective seeks to balance the goals of finding a large margin and ensuring low training classification error. The parameter $C$ controls the relative weighting between the two. In practice, the value for $C$ is often determined by cross validation.

## Dataset

We use the `p53 Mutants` {cite}`svm-lathrop2009p53` dataset from [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/188/p53+mutants). The dataset has 16,772 samples and each sample has 5,408 features assembled from biophysical simulations, and a binary label (transcriptonally competent, active p53 vs. cancerous, inactive p53) determined via in vivo assays. The goal is to use these 2D electrostatic and surface based as well as 3D distance based features to predict mutant p53 transcriptional activity. 

We download the dataset, and form training and test subsets with random 70-30 split. 

In [1]:
from urllib.request import urlretrieve
import zipfile, os
import pandas as pd

# set seed for reproducibility
seed = 0

# download the dataset
url = 'https://archive.ics.uci.edu/static/public/188/p53+mutants.zip'
file_Path = 'p53+mutants.zip'
urlretrieve(url, file_Path)

# extract datafile
with zipfile.ZipFile(file_Path, 'r') as zip_ref:
    zip_ref.extract('p53_old_2010.zip', '')
with zipfile.ZipFile('p53_old_2010.zip', 'r') as zip_ref:
    with zip_ref.open('p53_old_2010/K8.data') as f:
        # need to ignore last column because the datafile 
        # contains an extra comma at the end of each line
        df = pd.read_csv(f, usecols=range(5409), header=None, na_values='?')
os.remove('p53_old_2010.zip') # clean up extracted file

# remove samples with missing value
df = df.dropna()

# convert string labels to binary {-1, +1}
df[df.columns[-1]] = df.iloc[:,-1].map({'inactive': -1.0, 'active': 1.0})

# perform random split
train_df = df.sample(frac=0.7, random_state=seed)
test_df = df.drop(train_df.index)
X_train = train_df.iloc[:,:-1].to_numpy()
y_train = train_df.iloc[:,-1].to_numpy()
X_test = test_df.iloc[:,:-1].to_numpy()
y_test = test_df.iloc[:,-1].to_numpy()
del df, train_df, test_df

# get the shape of training samples
n, p = X_train.shape
n_test = X_test.shape[0]

## Fitting the model

To train the SVM classifier, we work with the dual problem
$$
\begin{aligned}
     \underset{\alpha}{\operatorname{maximize}} \quad& \sum_{i=1}^{n} \alpha_i - \frac{1}{2} \sum_{i,j}^{n} \alpha_i \alpha_j y_i y_j x_i^{\mathsf{T}} x_j \\
     \text{subject to} \quad& \sum_{i=1}^{n} \alpha_i y_i = 0 \\
     & 0 \leqslant \alpha_i \leqslant C, \, \forall i
\end{aligned}
$$

We see the objective involves the input features only via inner products. This enables us to use a more flexible variant of SVM that enlarges the feature space using basis expansion with kernel functions. The resulting classifier is capable of finding non-linear boundaries. The updated optimization problem is the following
$$
\begin{aligned}
     \underset{\alpha}{\operatorname{maximize}} \quad& \sum_{i=1}^{n} \alpha_i - \frac{1}{2} \sum_{i,j}^{n} \alpha_i \alpha_j y_i y_j \langle \phi(x_i), \, \phi(x_j) \rangle \\
     \text{subject to} \quad& \sum_{i=1}^{n} \alpha_i y_i = 0 \\
     & 0 \leqslant \alpha_i \leqslant C, \, \forall i
\end{aligned}
$$

where $\phi(\cdot)$ is the kernel function that transforms feature vectors. The above problem can be expressed more succinctly (as in the form presented by ), 
$$
\begin{aligned}
     \underset{\alpha}{\operatorname{minimize}} \quad& \frac{1}{2} \alpha^{\mathsf{T}} \operatorname{diag}(y) K \operatorname{diag}(y) \alpha - \vec{1}^{\mathsf{T}}\alpha \\
     \text{subject to} \quad& y^{\mathsf{T}}\alpha = 0 \\
     & 0 \leqslant \alpha_i \leqslant C, \, \forall i
\end{aligned}
$$
where $K_{i,j} = K(x_i, x_j) = \langle \phi(x_i), \, \phi(x_j) \rangle$. We use the radial basis function (RBF) kernel in this example, and thus $K(\cdot, \cdot)$ takes the form
$$
    K(x,x') = \exp(-\gamma \lVert x - x' \rVert_2^2)
$$
where $\gamma > 0$ is a parameter that sets the kernel width. A smaller value of $\gamma$ leads to more smooth decision boundaries. 

Here we use the [scikit-learn RBF kernel](https://scikit-learn.org/1.5/modules/generated/sklearn.metrics.pairwise.rbf_kernel.html) implementation and, precompute the Gram matrix $G \coloneqq \operatorname{diag}(y) K \operatorname{diag}(y)$ with kernel width set to $\gamma = 1/p$ (default value of the implementation). 

In [2]:
import jax.numpy as jnp
from sklearn.metrics.pairwise import rbf_kernel

# precompute kernel and Gram matrices
K = rbf_kernel(X_train)
G = jnp.diag(y_train) @ K @ jnp.diag(y_train)

We demonstrate how to set up NysADMM to train the SVM classifier. To apply NysADMM, we need to reformulate the problem and split the objective into separate components. One way of doing this is to define

- ``fun``: $\frac{1}{2} \alpha^{\mathsf{T}} \operatorname{diag}(y) K \operatorname{diag}(y) \alpha$
- ``reg_g``: $-\vec{1}^{\mathsf{T}}\alpha$
- ``reg_h``: $1_{\mathcal{C}}$ where $\mathcal{C} = \{\alpha \in \mathbb{R}^{n} \mid y^{\mathsf{T}}\alpha = 0, \, 0 \leqslant \alpha_i \leqslant C, \, \forall i\}$ is the intersection of a hyperplane and a hypercube

The proximal operator of the indicator function $1_{\mathcal{C}}$ is the projection onto $\mathcal{C}$, which is given by
$$
    \operatorname{prox}_{\lambda \cdot 1_{\mathcal{C}}}(z) = \Pi_{[0,C]}(z - \mu^{\ast}y)
$$
where $\Pi_{[0,C]}(\cdot)$ is the projection onto the hypercube
$$
    \big[\Pi_{[0,C]}(z)\big]_{j} = 
    \begin{cases}
        ~ 0 & \text{if}~ z_j < 0 \\
        ~ z_j & \text{if}~0 \leqslant z_j \leqslant C \\
        ~ C & \text{if}~ z_j > C \\
    \end{cases}
$$
and $\mu^{\ast}$ is the root of 
$$
    \varphi(\mu) \coloneqq y^{\mathsf{T}}\Pi_{[0,C]}(z - \mu y)
$$
We can use bisection method to find $\mu^{\ast}$. We implement these component functions along with related oracles and the aforementioned proximal operator.  

In [3]:
import jax

# define the objective `fun`
def fun(params, data): 
    return 0.5 * jnp.dot(params, data @ params)

# define the gradient oracle
def grad_fun(params, data): 
    return data @ params

# define the hvp oracle
def hvp_fun(params, vec, data): 
    return data @ vec

# define the smooth `reg_g`
def reg_g(params): 
    return -jnp.sum(params)

# define the proximal operator of the non-smooth `reg_h`
def prox_reg_h(point, scaling, C): 
    del scaling

    # define hypercube parameters
    lower = 0.0
    upper = C

    # define hyperplane parameters
    coeffs = y_train
    scalar = 0.0

    # define projection onto hypercube
    def hypercube_proj(x): 
        return jnp.minimum(jnp.maximum(x, lower), upper)

    # define bisection objective 
    def phi(mu):
        return jnp.dot(coeffs, hypercube_proj(point - mu * coeffs)) - scalar

    # identify bisection bracket
    bisect_lower = jax.lax.while_loop(lambda value: phi(value) < 0, lambda value: value * 2, -1)
    bisect_upper = jax.lax.while_loop(lambda value: phi(value) > 0, lambda value: value * 2, 1)

    # find optimal mu
    max_iter = 100
    atol = 1e-5
    rtol = 4 * jnp.finfo('float32').eps
    
    def cond_fun(values): 
        iter_num, l, u = values
        c = (l + u) / 2
        return (iter_num < max_iter) & (phi(c) != 0) & (jnp.abs((l - u) / 2) >= atol + rtol * jnp.abs(c))

    def body_fun(values): 
        iter_num, l, u = values
        c = (l + u) / 2
        r = (jnp.sign(phi(c)) * jnp.sign(phi(l)) + 1) / 2
        return iter_num + 1, (1 - r) * l + r * c, r * u + (1 - r) * c

    _, l, u = jax.lax.while_loop(cond_fun, body_fun, (0, bisect_lower, bisect_upper))
    mu = (l + u) / 2

    # compute projection
    y = hypercube_proj(point - mu * coeffs)
    
    return y

```{note}
Besides the built-in SketchyOpts proximal operations there are dedicated libraries for computing proximal mappings. For instance, [PyProx](https://pyproximal.readthedocs.io/en/stable/api/generated/pyproximal.projection.HyperPlaneBoxProj.html) implements the relevant projection that can be adapted for our use. 

However, we might need to make sure these external implementations are compatible with JAX. Here we choose to implement the proximal operator ourselves because `HyperPlaneBoxProj` from PyProx does not work well with JIT-compilation. 
```

We fit the model using NysADMM with sketch size $20$. As discussed in the NysADMM paper {cite}`svm-zhao2022nysadmm`, the Gram matrix $\operatorname{diag}(y) K \operatorname{diag}(y)$ is approximately low rank, and we would expect NysADMM to enjoy accelerated convergence in practice even if the effective dimension of the Hessian of the ADMM subproblem is greater than the sketch size. 

In [4]:
from sketchyopts.solver import NysADMM

# define optimizer parameters
sketch_size = 20
step_size = 500
maxiter = 100
init_params = jnp.zeros(n)
C = 1.0 / n

# run NysADMM to fit SVM classifier
opt = NysADMM(fun=fun, 
              grad_fun=grad_fun, 
              hvp_fun=hvp_fun, 
              reg_g=reg_g, 
              prox_reg_h=prox_reg_h, 
              step_size=step_size, 
              sketch_size=sketch_size, 
              maxiter=maxiter)
params, state = opt.run(init_params, G, prox_reg_h_params={'C': C})

To evaluate the test accuracy of the fitted classifier, we first recover optimal primal variables $\beta^{\ast}$ from the obtained optimal dual variables $\alpha^{\ast}$
$$
\begin{aligned}
    & \beta_1^{\ast} = \sum_{i=1}^{n} \alpha_i^{\ast} y_i \phi(x_i) \\
    & \beta_0^{\ast} = y_k - (\beta_1^{\ast})^{\mathsf{T}} \phi(x_k) \quad \text{for any}~k~\text{such that}~0 < \alpha_k^{\ast} < C
\end{aligned}
$$

For numerical stability, we take the average of all such $(x_k, y_k, \alpha_k^{\ast})$'s when computing $\beta_0^{\ast}$. 

Then we can make predictions on the unseen example $x$ using recovered $\beta^{\ast}$ by the following decision rule
$$
    \operatorname{sign}\big((\beta_1^{\ast})^{\mathsf{T}} \phi(x) + \beta_0^{\ast}\big)
    = \operatorname{sign}\bigg(\sum_{i=1}^{n} \alpha_i^{\ast} y_i K(x, x_i) + \beta_0^{\ast}\bigg)
$$
where $K(x, x_i) = \langle \phi(x), \phi(x_i) \rangle$. We now implement prediction function and compute the test accuracy. 

In [5]:
# compute new kernel matrix for testing
K_test = rbf_kernel(X_test, X_train)

# define prediction function
def predict(params, kernel): 
    # recover bias
    idx = jnp.argwhere(params[(0 < params) & (params < C)]).flatten()
    bias = jnp.mean(y_train[idx] - K[idx,:] @ (params * y_train))
    # make predictions
    preds = jnp.sign(kernel @ (params * y_train) + bias)
    return preds
    
# define test error evaluation function
def eval_test_error(preds): 
    return jnp.sum(0.5 * jnp.abs(y_test - preds)) / n_test

# output result
test_accuracy = 1.0 - eval_test_error(predict(params, K_test))
print('Test accuracy: {:.2f}%'.format(test_accuracy * 100))

Test accuracy: 99.28%


## Remarks

In this example, we see NysADMM is able to converge fast. Here we compare the training time to the [scikit-learn SVM](https://scikit-learn.org/dev/modules/generated/sklearn.svm.SVC.html) implementation which is based on LIBSVM {cite}`svm-chang2011libsvm`. As illustrated in the paper {cite}`svm-zhao2022nysadmm`, NysADMM is expected to have great performance when the Hessian of the ADMM subproblem is dense and approximately low rank. 

In [6]:
from sklearn.svm import SVC

# include precompute time for fair comparison
def solve_with_NysADMM(): 
    G = jnp.diag(y_train) @ rbf_kernel(X_train) @ jnp.diag(y_train)
    opt = NysADMM(fun=fun, 
              grad_fun=grad_fun, 
              hvp_fun=hvp_fun, 
              reg_g=reg_g, 
              prox_reg_h=prox_reg_h, 
              step_size=step_size, 
              sketch_size=sketch_size, 
              maxiter=maxiter)
    params, state = opt.run(init_params, G, prox_reg_h_params={'C': C})

# use same hyerparameters as NysADMM
def solve_with_SVC(): 
    clf = SVC(C=1.0, gamma='auto')
    clf.fit(X_train, y_train)

print("NysADMM:")
%timeit solve_with_NysADMM()

print("SVC:")
%timeit solve_with_SVC()

NysADMM:
13.3 s ± 103 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
SVC:
1min 6s ± 422 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## References

```{bibliography}
:labelprefix: SVM
:keyprefix: svm-
```